<a href="https://colab.research.google.com/github/Sgraham6/Sean-Graham-Projects/blob/main/Laptop-Regression/Regression_Code_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
#import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.stats.diagnostic as diag
import statsmodels.robust.robust_linear_model as rlm
from fractions import Fraction
from statsmodels.stats.stattools import durbin_watson, jarque_bera
from statsmodels.tools.tools import pinv_extended
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RepeatedKFold #, cross_validate
from sklearn import metrics
from sklearn.linear_model import ElasticNet, ElasticNetCV
from scipy.stats import boxcox

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Laptop_Data_Reduced = pd.read_csv('/content/drive/MyDrive/Personal Files/Regression Practice/Laptop_Data_Reduced.csv')
Laptop_Data_Reduced.head()

,Company_Asus,Company_Dell,Company_HP,Company_Lenovo,Company_MSI,Company_Other,Company_Toshiba,OpSys_Linux,OpSys_No OS,OpSys_Windows 7,...,MemoryGB1_Other,MemoryType1_HDD,ScreenResolution_1600x900,ScreenResolution_2560x1440,ScreenResolution_3840x2160,ScreenDisplayType_None,ScreenDisplayType_Quad HD+,ScreenIPSPanel_Yes,ScreenTouchscreen_Yes,Price
0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,30636.000
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,21312.000
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,79653.600
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,41025.600
4,0,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,20986.992


In [ ]:
# Split and normalize the data, transfrom y variable

X = Laptop_Data_Reduced.drop('Price', axis=1)
y = Laptop_Data_Reduced['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

X_train_constant = sm.add_constant(X_train)
X_test_constant = sm.add_constant(X_test)

In [ ]:
y_train_temp, optimal_lambda = boxcox(y_train)
print(optimal_lambda)

0.09457515233235643


In [ ]:
def function(array):
  result = 0.9 * (array - 1)**3 + 1
  inverse = abs((array - 1) / 0.9)**(1/3) * np.where((array) < 1, -1, 1) + 1
  return result, inverse

def elastic_net_search (alpha_values, l1_ratio_values):
  elastic_net_cv = ElasticNetCV(alphas = alpha_values, l1_ratio = l1_ratio_values, cv = cv)
  elastic_net_cv.fit(X_train, j)
  return elastic_net_cv.alpha_, elastic_net_cv.l1_ratio_

transformation_values_factions_and_whole = [-2, -3/2, -1, -1/2, -1/3, -1/4, -1/5, 0, 1/5, 1/4, 1/3, 1/2, 1, 3/2, 2]
transformation_values_decimals = [optimal_lambda]
transformation_values = transformation_values_factions_and_whole + transformation_values_decimals

#transformation_values_table = pd.DataFrame(columns = ['Transformation', 'Best Alpha', 'Best L1'])
transformation_values_table = pd.read_csv('/content/drive/MyDrive/Personal Files/Regression Practice/Transformation_Values_Table_4.csv')
transformation_values_table = transformation_values_table.replace('ln(x)', 'x^0').replace('ln(ln(x))', 'ln^0(x)')

for i in transformation_values:
  displayed_value = str(round(i, 2)) if i == optimal_lambda else \
    (str(Fraction(i).limit_denominator(10)) if i in transformation_values_factions_and_whole else str(i))
  if str('x^' + displayed_value) not in [str(element) for element in transformation_values_table['Transformation'].tolist()]:
    normal, compound = (np.log(y_train), np.log(np.log(y_train))) if i == 0 else (np.power(y_train, i), np.power(np.log(y_train), i))
    for j in normal, compound:
      transformation_name = str('x^' + displayed_value) if j is normal else str('ln^' + displayed_value + '(x)')
      cv = RepeatedKFold(n_splits = 10, n_repeats = 5, random_state = 1)

      alpha_values_1, l1_ratio_values_1 = [5e-3, 5e-2, 5e-1, 5], function(np.arange(0.05, 1, 0.1))[0]
      best_alpha_1, best_l1_ratio_1 = elastic_net_search(alpha_values_1, l1_ratio_values_1)

      alpha_values_2 = (np.arange(best_alpha_1 / 5, best_alpha_1 * 2.0000001, best_alpha_1 / 5))
      l1_ratio_values_2 = function(np.arange(function(best_l1_ratio_1)[1] - 0.05, function(best_l1_ratio_1)[1] + 0.06, 0.01))[0]
      best_alpha_2, best_l1_ratio_2 = elastic_net_search(alpha_values_2, l1_ratio_values_2)

      new_row_data_transformation = [transformation_name, best_alpha_2, best_l1_ratio_2]
      transformation_values_table.loc[len(transformation_values_table)] = new_row_data_transformation

In [ ]:
def organize (transformation_value):
  number_component_string = re.search("-?(\d+\/\d+|\d*\.?\d+)", transformation_value)[0]
  number_component = eval(number_component_string)
  letter_component = transformation_value.replace('ln', 'y')[0]
  return (letter_component, number_component)

#transformation_values_table = transformation_values_table.drop([0], axis=0)
#transformation_values_table.loc[0, 'Lambda'] = ''

transformation_values_table = transformation_values_table.sort_values(by='Transformation', key = lambda x: x.apply(organize))
transformation_values_table = transformation_values_table.replace('x^0', 'ln(x)').replace('ln^0(x)', 'ln(ln(x))') \
  .drop(transformation_values_table[transformation_values_table['Transformation'] == 'ln^1(x)'].index, axis=0)
transformation_values_table.reset_index(drop=True, inplace=True)
display(transformation_values_table)

,Transformation,Best Alpha,Best L1
0,x^-2,10.000,0.100000
1,x^-3/2,10.000,0.100000
2,x^-1,10.000,0.100000
3,x^-1/2,10.000,0.100000
4,x^-1/3,0.001,0.100000
5,x^-1/4,0.001,0.100000
6,x^-1/5,0.001,0.100000
7,ln(x),0.001,0.100000
8,x^0.09,0.001,0.100000
9,x^1/5,0.001,0.100000


In [ ]:
transformation_values_table.to_csv('/content/drive/MyDrive/Personal Files/Regression Practice/Transformation_Values_Table_4.csv', index=False)

In [ ]:
summary_table = pd.DataFrame(columns = ['Transformation', 'Model', 'Best Alpha', 'Best L1', 'Adj R2', 'Jarque Bera', 'Durbin Watson', 'Norm Train', 'Norm Test', 'Hetero Train'])
#summary_table = pd.read_csv('/content/drive/MyDrive/Personal Files/Regression Practice/Summary_Table_4.csv')

In [ ]:
# Define Functions
def alpha_and_l1 (i):
  best_alpha = transformation_values_table.loc[transformation_values_table['Transformation'] == i, 'Best Alpha'].iloc[0]
  best_l1_ratio = transformation_values_table.loc[transformation_values_table['Transformation'] == i, 'Best L1'].iloc[0]
  return best_alpha, best_l1_ratio

def transform (array, i):
  if str(round(optimal_lambda, 2)) in i:
    i = i.replace(str(round(optimal_lambda, 2)), str(optimal_lambda))

  if i == 'ln(x)':
    return np.log(array)
  elif i == 'ln(ln(x))':
    return np.log(np.log(array))
  elif 'x^' in i:
    value = organize(i)[1]
    return np.power(array, value)
  else:
    value = organize(i)[1]
    return np.power(np.log(array), value)

def model_reg_and_summary_base (model):
  model_regularized_elastic = model.fit_regularized(method = 'elastic_net', alpha = best_alpha, L1_wt = best_l1_ratio)
  pinv_wexog,_ = pinv_extended(model.wexog)
  model_regularized_elastic_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
  summary_base_elastic = sm.regression.linear_model.OLSResults(model, model_regularized_elastic.params, model_regularized_elastic_cov_params)
  return model_regularized_elastic, summary_base_elastic

def obtain_residuals (model_or_reg_model):
  predictions_train = model_or_reg_model.predict(X_train_constant)
  residuals_train = y_train_trans - predictions_train
  predictions_test = model_or_reg_model.predict(X_test_constant)
  residuals_test = y_test_trans - predictions_test
  return residuals_train, residuals_test

def pseudo_adj_rsquared (model_or_summary_base):
  pseudo_r2 = model_or_summary_base.pseudo_rsquared(kind='mcf')
  n, p = len(model_or_summary_base), len(model_or_summary_base.params) - 1
  pseudo_adj_r2 = 1 - ((n - 1) / (n - p - 1)) * (1 - pseudo_r2)

def new_row (model_or_summary_base, residuals_train, residuals_test, name):
  adj_r2 = model_or_summary_base.rsquared_adj if hasattr(model_or_summary_base, 'rsquared_adj') else pseudo_adj_rsquared(model_or_summary_base)
  jb = jarque_bera(residuals_train)[0]
  dw = durbin_watson(residuals_train)
  norm_train = diag.normal_ad(residuals_train)[1]
  norm_test = diag.normal_ad(residuals_test)[1]
  hetero_train = diag.het_white(residuals_train, X_train_constant)[1]
  new_row_data = [i, name, best_alpha, best_l1_ratio, adj_r2, jb, dw, norm_train, norm_test, hetero_train]
  summary_table.loc[len(summary_table)] = new_row_data
  return summary_table

In [ ]:
# OLS
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'OLS')).any():
    best_alpha, best_l1_ratio = '', ''
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols = sm.OLS(y_train_trans, X_train_constant).fit()
    residuals_train_ols, residuals_test_ols = obtain_residuals(model_ols)
    summary_table = new_row(model_ols, residuals_train_ols, residuals_test_ols, 'OLS')

# OLS Elastic
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'OLS Elastic')).any():
    best_alpha, best_l1_ratio = alpha_and_l1(i)
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols_elastic = sm.OLS(y_train_trans, X_train_constant)
    model_regularized_ols_elastic, summary_base_ols_elastic = model_reg_and_summary_base(model_ols_elastic)
    residuals_train_ols_elastic, residuals_test_ols_elastic = obtain_residuals(model_regularized_ols_elastic)
    summary_table = new_row(summary_base_ols_elastic, residuals_train_ols_elastic, residuals_test_ols_elastic, 'OLS Elastic')

In [ ]:
# Perform weighted least squares
# some code was taken from GeeksforGeeks

# WLS1
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'WLS1')).any():
    best_alpha, best_l1_ratio = '', ''
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols = sm.OLS(y_train_trans, X_train_constant).fit()
    residuals_train_ols, residuals_test_ols = obtain_residuals(model_ols)
    weights_train1 = 1 / sm.OLS(abs(residuals_train_ols), model_ols.predict(X_train_constant)).fit().fittedvalues**2
    weights_test1 = 1 / sm.OLS(abs(residuals_test_ols), model_ols.predict(X_test_constant)).fit().fittedvalues**2

    model_wls1 = sm.WLS(y_train_trans, X_train_constant, weights = weights_train1).fit()
    residuals_train_wls1, residuals_test_wls1 = obtain_residuals(model_wls1)
    residuals_train_wls1_weighted, residuals_test_wls1_weighted = residuals_train_wls1 * np.sqrt(weights_train1), residuals_test_wls1 * np.sqrt(weights_test1)
    summary_table = new_row(model_wls1, residuals_train_wls1_weighted, residuals_test_wls1_weighted, 'WLS1')

# WLS1 Elastic
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'WLS1 Elastic')).any():
    best_alpha, best_l1_ratio = alpha_and_l1(i)
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols_elastic = sm.OLS(y_train_trans, X_train_constant)
    model_regularized_ols_elastic, summary_base_ols_elastic = model_reg_and_summary_base(model_ols_elastic)
    residuals_train_ols_elastic, residuals_test_ols_elastic = obtain_residuals(model_regularized_ols_elastic)
    weights_train1_elastic = 1 / sm.OLS(abs(residuals_train_ols_elastic), model_regularized_ols_elastic.predict(X_train_constant)).fit().fittedvalues**2
    weights_test1_elastic = 1 / sm.OLS(abs(residuals_test_ols_elastic), model_regularized_ols_elastic.predict(X_test_constant)).fit().fittedvalues**2

    model_wls1_elastic = sm.WLS(y_train_trans, X_train_constant, weights = weights_train1_elastic)
    model_regularized_wls1_elastic, summary_base_wls1_elastic = model_reg_and_summary_base(model_wls1_elastic)
    residuals_train_wls1_elastic, residuals_test_wls1_elastic = obtain_residuals(model_regularized_wls1_elastic)
    residuals_train_wls1_elastic_weighted, residuals_test_wls1_elastic_weighted = residuals_train_wls1_elastic * np.sqrt(weights_train1_elastic), residuals_test_wls1_elastic * np.sqrt(weights_test1_elastic)
    summary_table = new_row(summary_base_wls1_elastic, residuals_train_wls1_elastic_weighted, residuals_test_wls1_elastic_weighted, 'WLS1 Elastic')

In [ ]:
# Perform weighted least squares
# some code was taken from GeeksforGeeks

# WLS2
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'WLS2')).any():
    best_alpha, best_l1_ratio = '', ''
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols = sm.OLS(y_train_trans, X_train_constant).fit()
    residuals_train_ols, residuals_test_ols = obtain_residuals(model_ols)
    weights_train2 = 1 / sm.OLS(residuals_train_ols**2, model_ols.predict(X_train_constant)).fit().fittedvalues**2
    weights_test2 = 1 / sm.OLS(residuals_test_ols**2, model_ols.predict(X_test_constant)).fit().fittedvalues**2

    model_wls2 = sm.WLS(y_train_trans, X_train_constant, weights = weights_train2).fit()
    residuals_train_wls2, residuals_test_wls2 = obtain_residuals(model_wls2)
    residuals_train_wls2_weighted, residuals_test_wls2_weighted = residuals_train_wls2 * np.sqrt(weights_train2), residuals_test_wls2 * np.sqrt(weights_test2)
    summary_table = new_row(model_wls2, residuals_train_wls2_weighted, residuals_test_wls2_weighted, 'WLS2')

# WLS2 Elastic
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'WLS2 Elastic')).any():
    best_alpha, best_l1_ratio = alpha_and_l1(i)
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols_elastic = sm.OLS(y_train_trans, X_train_constant)
    model_regularized_ols_elastic, summary_base_ols_elastic = model_reg_and_summary_base(model_ols_elastic)
    residuals_train_ols_elastic, residuals_test_ols_elastic = obtain_residuals(model_regularized_ols_elastic)
    weights_train2_elastic = 1 / residuals_train_ols_elastic**2
    weights_test2_elastic = 1 / residuals_test_ols_elastic**2

    model_wls2_elastic = sm.WLS(y_train_trans, X_train_constant, weights = weights_train2_elastic)
    model_regularized_wls2_elastic, summary_base_wls2_elastic = model_reg_and_summary_base(model_wls2_elastic)
    residuals_train_wls2_elastic, residuals_test_wls2_elastic = obtain_residuals(model_regularized_wls2_elastic)
    residuals_train_wls2_elastic_weighted, residuals_test_wls2_elastic_weighted = residuals_train_wls2_elastic * np.sqrt(weights_train2_elastic), residuals_test_wls2_elastic * np.sqrt(weights_test2_elastic)
    summary_table = new_row(summary_base_wls2_elastic, residuals_train_wls2_elastic_weighted, residuals_test_wls2_elastic_weighted, 'WLS2 Elastic')

In [ ]:
# Robust Linear Models
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'RLM')).any():
    best_alpha, best_l1_ratio = '', ''
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_rlm = sm.RLM(y_train_trans, X_train_constant, M=sm.robust.norms.TukeyBiweight()).fit()
    residuals_train_rlm, residuals_test_rlm = obtain_residuals(model_rlm)
    summary_table = new_row(model_rlm, residuals_train_rlm, residuals_test_rlm, 'RLM')

In [ ]:
# Perform GLS
# Some code taken from goolge ai

# GLS
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'GLS')).any():
    best_alpha, best_l1_ratio = '', ''
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols = sm.OLS(y_train_trans, X_train_constant).fit()
    residuals_train_ols, residuals_test_ols = obtain_residuals(model_ols)
    var_model_train = sm.OLS(residuals_train_ols**2, X_train_constant).fit()
    var_prediction_train = var_model_train.predict(X_train_constant)
    var_estimation_train = np.maximum(var_prediction_train, 0.0001)
    sigma_train = np.diag(var_estimation_train)

    model_gls = sm.GLS(y_train_trans, X_train_constant, sigma = sigma_train).fit()
    residuals_train_gls, residuals_test_gls = obtain_residuals(model_gls)
    summary_table = new_row(model_gls, residuals_train_gls, residuals_test_gls, 'GLS')

# GLS Elastic
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'GLS Elastic')).any():
    best_alpha, best_l1_ratio = alpha_and_l1(i)
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols_elastic = sm.OLS(y_train_trans, X_train_constant)
    model_regularized_ols_elastic, summary_base_ols_elastic = model_reg_and_summary_base(model_ols_elastic)
    residuals_train_ols_elastic, residuals_test_ols_elastic = obtain_residuals(model_regularized_ols_elastic)
    var_model_train_elastic = sm.OLS(residuals_train_ols_elastic**2, X_train_constant).fit()
    var_prediction_train_elastic = var_model_train_elastic.predict(X_train_constant)
    var_estimation_train_elastic = np.maximum(var_prediction_train_elastic, 0.0001)
    sigma_train_elastic = np.diag(var_estimation_train_elastic)

    model_gls_elastic = sm.GLS(y_train_trans, X_train_constant, sigma = sigma_train_elastic)
    model_regularized_gls_elastic, summary_base_gls_elastic = model_reg_and_summary_base(model_gls_elastic)
    residuals_train_gls_elastic, residuals_test_gls_elastic = obtain_residuals(model_regularized_gls_elastic)
    summary_table = new_row(summary_base_gls_elastic, residuals_train_gls_elastic, residuals_test_gls_elastic, 'GLS Elastic')

In [ ]:
# GLM
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'GLM')).any():
    best_alpha, best_l1_ratio = '', ''
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_glm = sm.GLM(y_train_trans, X_train_constant, family = sm.families.Gamma(link = sm.families.links.Log())).fit()
    residuals_train_glm, residuals_test_glm = obtain_residuals(model_glm)
    summary_table = new_row(model_glm, residuals_train_glm, residuals_test_glm, 'GLM')

In [ ]:
# GLM Weighted
for i in transformation_values_table['Transformation']:
  if not ((summary_table['Transformation'] == i) & (summary_table['Model'] == 'GLM Weighted')).any():
    best_alpha, best_l1_ratio = '', ''
    y_train_trans, y_test_trans = transform(y_train, i), transform(y_test, i)

    model_ols = sm.OLS(y_train_trans, X_train_constant).fit()
    residuals_train_ols, residuals_test_ols = obtain_residuals(model_ols)
    weights_train2 = 1 / sm.OLS(residuals_train_ols**2, model_ols.predict(X_train_constant)).fit().fittedvalues**2
    weights_test2 = 1 / sm.OLS(residuals_test_ols**2, model_ols.predict(X_test_constant)).fit().fittedvalues**2

    model_glm2 = sm.GLM(y_train_trans, X_train_constant, family = sm.families.Gamma(link = sm.families.links.Log()), var_weights = weights_train2).fit()
    residuals_train_glm2, residuals_test_glm2 = obtain_residuals(model_glm2)
    residuals_train_glm2_weighted, residuals_test_glm2_weighted = residuals_train_glm2 * np.sqrt(weights_train2), residuals_test_glm2 * np.sqrt(weights_test2)
    summary_table = new_row(model_glm2, residuals_train_glm2_weighted, residuals_test_glm2_weighted, 'GLM Weighted')

/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)


In [ ]:
#summary_table = summary_table.drop([20], axis=0)

#summary_table.loc[0, 'Lambda'] = ''

#summary_table = summary_table.sort_values(by='Transformation')
#summary_table.reset_index(drop=True, inplace=True)

display(summary_table.tail(40))

,Transformation,Model,Best Alpha,Best L1,Adj R2,DW,Norm Train,Norm Test,Hetero Train
140,1,GLS,NaN,NaN,0.942269,1.871355,0.000000e+00,2.490782e-23,9.871095e-16
141,1.5,GLS,NaN,NaN,0.875063,1.972918,0.000000e+00,0.000000e+00,1.101679e-18
142,2,GLS,NaN,NaN,0.831675,2.010586,0.000000e+00,0.000000e+00,1.106815e-19
143,3,GLS,NaN,NaN,0.778339,2.010169,0.000000e+00,0.000000e+00,1.317327e-20
144,BoxCox,GLS Elastic,0.001,0.0,0.978835,1.566714,1.539804e-04,4.597661e-01,2.653986e-16
145,Ln,GLS Elastic,0.001,0.0,0.96253,1.853061,1.768328e-04,6.706079e-01,2.357993e-12
146,-3,GLS Elastic,0.01,0.0,0.617129,2.004374,0.000000e+00,0.000000e+00,5.658922e-05
147,-2,GLS Elastic,0.01,0.0,0.730991,1.993963,0.000000e+00,1.522421e-23,3.274427e-09
148,-1.5,GLS Elastic,0.01,0.0,0.780295,1.978848,0.000000e+00,2.058395e-12,2.078622e-11
149,-1,GLS Elastic,0.001,0.0,0.826336,1.941570,3.907477e-25,5.652470e-04,1.671400e-13


In [ ]:
summary_table.to_csv('/content/drive/MyDrive/Personal Files/Regression Practice/Summary_Table_4.csv', index=False)

In [ ]:
#fix the outline, consider removing appropriate variables and add this in outline somewhere

# Consider GLM (different assumptions from linear regression)  - do a histogram of errors first to show gamma pattern
# In future, see visuals in conjunction with table values and decide on something earlier; this time, choose the most ideal value and get the visuals for that
    # Shave down the table (ie; norm train != 0) under a new name and decide from there
# do plots for what you decide - maybe change "lm" before metrics to something else
# see tabs for next steps
# Save the model somehow
# Add comments and text explaining everything / make documents readable

# maybe add code for the sake of readability
# address endogeneirty with actual person when time
# address how saving and loading from google drive works when published
# See other people's github portfolios
# Upload final summary table to Github when ready

In [ ]:
# based on final selection, try swapping ln with log10 to see if it improves the log likelihood, AIC, and BIC
  #make another table for these values, include r2 for checking purposes, include selected model (type of regression and powers) in title
  #only code for one of the below for this document; can code for others in other regression projects
  #for ln(ln), try swapping first, second, and both ln values with ln(10)
  #for ln, try log10, also try taking ln and log 10 of the powers (not powers of the ln/log10; extract powers from a previous table) and see how it affects the three values
  #for plain power values, don't change
  #for ln power values, try  doing log10 instead in the inner function

# get a full summary for anything that passes all checks
# Try robust standard errors (see tab)

# Add pseudo rsquared to models that don't have it (see below for the glm one) - test it out
# Fix the transfrom function
# Fix fractions, fix the fact that x^0 reruns, make sure all values don't rerun

In [ ]:
# Check other four model assumptions:
  # Linearity - not necessary (no reamining conitnuous variables)
  # Independent Residuals - Durbin Watson value looks good
  #
  #

#https://datascience.stackexchange.com/questions/27615/should-we-apply-normalization-to-test-data-as-well

In [ ]:
x = model_glm.pseudo_rsquared()

n = len(y_train_trans)
p = 48
r2_adj = 1 - ((n - 1) / (n - p - 1)) * (1 - x)
print(r2_adj)
print(x)

0.9567628595998534
0.9589848967938652
